In [1]:
import pandas as pd
import usaddress
import os
import numpy as np

<h2>Load all watermains</h2>

In [2]:
all_mains = pd.read_csv('../data/raw/All_Watermains_Attributes.csv', encoding='latin-1').drop_duplicates()

In [3]:
all_mains['INSTALLDAT'] = pd.to_datetime(all_mains['INSTALLDAT'], format='mixed')

<h2>Load soil ph</h2>

In [4]:
soil_ph = pd.read_csv("../data/raw/Watermain_Attributes_Soil_Ph.csv")

In [5]:
soil_ph['INSTALLDAT'] = pd.to_datetime(soil_ph['INSTALLDAT'], format='mixed')

<h2>Load broken mains reports</h2>

In [6]:
reports = pd.read_csv("../data/raw/Watermain_Breaks_Reporting.csv")

<h2>Merge datsets</h2>

In [7]:
#'INSTALLDAT' & 'LOCATION' also common keys, but seem to loose too many instances
all_mains_broken = soil_ph[['FACILITYID', 'WORKORDERI']].merge(all_mains, on = ['FACILITYID'])

In [8]:
all_mains_broken = all_mains_broken.merge(reports, left_on = 'WORKORDERI', right_on = 'Work Order ID')

Add a flag to the all_mains dataframe for if a pipe has ever broken

In [9]:
broken_facility_ids = all_mains_broken['FACILITYID'].to_list()

In [22]:
all_mains['break_status'] = all_mains['FACILITYID'].apply(lambda s: np.where(s in broken_facility_ids, 'has broken', 'has never broken'))

Get the dates of all the breaks

In [11]:
all_mains_broken['Date Initiated'] = pd.to_datetime(all_mains_broken['Date Initiated'])
break_dates_by_facility_id = all_mains_broken.groupby('FACILITYID')['Date Initiated'].apply(list).reset_index()
break_dates_by_facility_id = break_dates_by_facility_id.rename(columns = {'Date Initiated': 'all_breaks'})

In [13]:
first_and_last_break = all_mains_broken.groupby('FACILITYID')['Date Initiated'].agg(['min', 'max']).reset_index()
first_and_last_break = first_and_last_break.rename(columns = {'min': 'first_break', 'max': 'most_recent_break'})

Add date info to main dataframe

In [14]:
all_mains_broken = all_mains_broken.merge(first_and_last_break, on = 'FACILITYID')
all_mains_broken = all_mains_broken.merge(break_dates_by_facility_id, on = 'FACILITYID')

Set up final dataframe

In [17]:
cols_to_keep = list(all_mains.columns)
cols_to_keep.extend(['all_breaks', 'first_break', 'most_recent_break'])

In [20]:
all_mains_broken['break_status'] = 'has broken'
all_mains_broken = all_mains_broken[cols_to_keep]

In [25]:
never_broken = all_mains[all_mains['break_status'] == 'has never broken']

In [28]:
df = pd.concat([never_broken, all_mains_broken])

In [29]:
df.to_csv("../data/transformed/watermain_breaks_dataset.csv", index = False)